---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 
          'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois',
          'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine',
          'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey',
          'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas',
          'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 
          'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 
          'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 
          'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands',
          'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
import re
def get_list_of_university_towns():
    st_twn = pd.read_csv('university_towns.txt', header = None, sep='\n', dtype = str)
    st_twn = st_twn[0].str.split('(', expand=True)
    st_twn = st_twn.drop(st_twn.columns[[1,2,3]], axis = 1)
    st_twn.columns = ['All_Lst']
    st_twn = st_twn['All_Lst'].str.extract(r'(?P<State>.*[[edit]])|(?P<RegionName>.*)')
    st_twn['State'].fillna(method = 'ffill', inplace = True)
    st_twn['State'] = st_twn['State'].str.split('[', expand=True)
    st_twn['State'].apply(lambda x:re.sub('[\\[].*[\\\\n]','',x))
    st_twn['RegionName'] = st_twn['RegionName'].str.strip()
    #st_twn = st_twn[~st_twn.RegionName.str.contains(':', na = False)]
    st_twn = st_twn.dropna().reset_index(drop=True)
    
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    return st_twn 
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [4]:
GDP = pd.read_excel('gdplev.xls', index_col=None, skiprows = 7)
GDP = GDP.iloc[:,4:7].rename(columns = {'Unnamed: 4':'Qtr', 'Unnamed: 6':'GDP_cv'}).drop(['Unnamed: 5'], axis = 1)
GDP = GDP[~GDP.Qtr.str.contains('19', na = False)]
GDP.groupby('Qtr')
GDP['Dif'] = GDP.GDP_cv.diff()

def get_recession_start():    
    for i in range(len(GDP.Dif)):
        if GDP.Dif.iloc[i] < 0 and GDP.Dif.iloc[i+1] < GDP.Dif.iloc[i]:
            return GDP.Qtr.iloc[i]
    
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    GDP['Dif'] = GDP['Dif']
    for i in range(len(GDP.Dif)):
        if GDP.Dif.iloc[i] < 0 and GDP.Dif.iloc[i+1] < GDP.Dif.iloc[i]:
            while GDP.Dif.iloc[i] < 0:
                i = i+1
                if GDP.Dif.iloc[i+1] > 0 and GDP.Dif.iloc[i+2] > GDP.Dif.iloc[i+1]:
                    return GDP.Qtr.iloc[i+2]
    
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    GDPbtm = GDP.loc[GDP.Qtr.between('2008q3', '2009q4', inclusive = True)]
    for i in range(len(GDPbtm)):
        if GDPbtm.GDP_cv.iloc[i] == GDPbtm.GDP_cv.min():
            return GDPbtm.Qtr.iloc[i]
    
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
get_recession_bottom()

'2009q2'

In [7]:
def convert_housing_data_to_quarters():
    HDat = pd.read_csv('City_Zhvi_AllHomes.csv')
    HDat.drop(HDat.iloc[:, np.r_[0, 3:51]], inplace = True, axis = 1)
    for acr, stname in states.items():
         HDat.State = HDat.State.str.replace(acr, stname)
    HDat = HDat.set_index(['State','RegionName'])
    HDat.applymap(lambda x: np.nan if x == ' ' else x)
    HDat = HDat.groupby((np.arange(len(HDat.columns))//3), axis = 1).mean()
    HDat.columns = [pd.date_range(periods = len(HDat.columns), end = '2016-09-30', freq = '3M').to_period('Q')]
    HDat.columns = HDat.columns.strftime('%Yq%q')

    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    return HDat.head(10)
convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0
Nevada,Las Vegas,132600.000000,134366.666667,135400.000000,137000.000000,139533.333333,141733.333333,143366.666667,146133.333333,149333.333333,150933.333333,...,170066.666667,173400.000000,175466.666667,177500.000000,181600.000000,186766.666667,190633.333333,194600.000000,197200.000000,199950.0
California,San Diego,222900.000000,234366.666667,245433.333333,256033.333333,267200.000000,276266.666667,284500.000000,291933.333333,301233.333333,312866.666667,...,480200.000000,489033.333333,496433.333333,503366.666667,512066.666667,519766.666667,525466.666667,529333.333333,536233.333333,539750.0
Texas,Dallas,84466.666667,83866.666667,84866.666667,87833.333333,89733.333333,89300.000000,89066.666667,90900.000000,92566.666667,93800.000000,...,106633.333333,108900.000000,111533.333333,113700.000000,121133.333333,128566.666667,134600.000000,140500.000000,144600.000000,149300.0
California,San Jose,374266.666667,406566.666667,431866.666667,455500.000000,470666.666667,470200.000000,456800.000000,445566.666667,441433.333333,457766.666667,...,679400.000000,697033.333333,714933.333333,731433.333333,756733.333333,776400.000000,789133.333333,803600.000000,818933.333333,822200.0


In [8]:
def run_ttest():
    s = get_recession_start()
    HDat = convert_housing_data_to_quarters().reset_index()
    strt = HDat.columns.get_loc(s)-1
    strt = HDat.iloc[:, strt].name
    bttm = get_recession_bottom()
    univ_twn = get_list_of_university_towns()
    
    Twns = pd.merge(HDat, univ_twn, on = univ_twn.columns.tolist(), indicator = '_mrg', how = 'outer')
    univ_twn = Twns[Twns['_mrg']=='both']
    non_univ_twn = Twns[Twns['_mrg'] != 'both']
    
    univ_twn['r'] = univ_twn[strt] / univ_twn[bttm]
    non_univ_twn['r'] = non_univ_twn[strt] / non_univ_twn[bttm]
    
    s, pv = ttest_ind(univ_twn.r,non_univ_twn.r, nan_policy = 'omit')
    
    if univ_twn['r'].mean() < non_univ_twn['r'].mean():
        better = 'university town'
    else:
        better = 'non-university town'
    
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return (pv<0.01, pv, better)
    
run_ttest()

(False, 0.285466335780933, 'non-university town')